In [8]:
from time import time
import keras
import os
import sys
sys.path.append('../')
from utils.utils import smooth_labels, CLRScheduler, LinearDecayScheduler, WarmUpCosineDecayScheduler
from utils.codification_cnn import CNNLayer, NNLayer, ChromosomeCNN, FitnessCNN, FitnessCNNParallel

from utils.codification_skipc import FitnessSkip, Connections, ChromosomeSkip
from utils.datamanager import DataManager
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
%matplotlib inline

from keras import Input, Model
from keras.layers import Conv2D, PReLU, LeakyReLU, Dropout,SpatialDropout2D
from keras.layers import MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.layers.merge import concatenate

from keras.layers import SeparableConv2D, DepthwiseConv2D, LocallyConnected2D


def get_chromosome_from_file(filename):
    cnn_layers = []
    nn_layers = []
    connections = []
    with open(filename, 'r') as f:
        for line in f:
            params = line.split('|')            
            if 'CNN' == params[0]:
                filters = int(params[1].split(':')[1])
                kernel = literal_eval(params[2].split(':')[1])
                activation = params[3].split(':')[1]
                dropout = float(params[4].split(':')[1])
                maxpool = bool(int(params[5].split(':')[1]))
                cnn_layers.append(CNNLayer(filters, kernel, activation, dropout, maxpool))
            elif 'NN' == params[0]:
                units = int(params[1].split(':')[1])
                activation = params[2].split(':')[1]
                dropout = float(params[3].split(':')[1])
                nn_layers.append(NNLayer(units, activation, dropout))
            else:
                try:
                    local_connections = [int(el) for el in params[0].split("\n")[0]]
                    if len(local_connections) > 0:
                        connections.append(local_connections)
                except ValueError:
                    pass
        max_len = np.max([len(e) for e in connections])
        matrix = np.array([el + [0] * (max_len - len(el)) for el in connections])
        connections = Connections(matrix)
    return ChromosomeSkip(cnn_layers, nn_layers, connections)


def show_result(history, metric='acc'):
        color = np.array([[31, 119, 180], [255, 127, 14]]) / 255.
        try:
            epochs = np.linspace(0, len(history.history['acc']) - 1, len(history.history['acc']))
            argmax_val = np.argmax(history.history['val_%s' % metric])
            plt.plot(epochs, history.history['val_%s' % metric], label='validation', color=color[0], alpha=0.5)
            plt.plot(epochs, median_filter(history.history['val_%s' % metric]), color=color[0])
            plt.scatter(epochs[argmax_val], history.history['val_%s' % metric][argmax_val],
            label='max val_%s %0.4f' % (metric, history.history['val_%s' % metric][argmax_val]), c='r')
        except KeyError:
            pass
        plt.plot(epochs, history.history[metric], label='train', color=color[1], alpha=0.5)
        plt.plot(epochs, median_filter(history.history[metric]), color=color[1])


        plt.legend()
        plt.xlabel('Epochs')
        #plt.xticks([20, 40, 60, 80, 120, 160, 180])
        plt.grid()
        plt.ylabel(metric)
        plt.show()
        
def median_filter(v, size=5):
    stride = (size - 1) // 2
    filtered_v = []
    for i in range(len(v)):
        if i < stride or i + stride == len(v):
            filtered_v.append(v[i])
        else:
            filtered_v.append(np.mean(v[i-stride:i + stride + 1]))
    return filtered_v

def select_conv(inp, filters, k_size, type_, act=None):
    if type_ == 'sep':
        return SeparableConv2D(filters, k_size, activation=act_, padding='same')(inp)
    elif type_ == 'depw':
        return DepthwiseConv2D(filters, k_size, activation=act_, padding='same')(inp)
    elif type_ == 'loc':
        return LocallyConnected2D(filters, k_size, activation=act_, padding='same')(inp)
    else:
        print("%s conv type is not defined" % type_)

class FitnessSkip_v2(FitnessSkip):
    maxpool_overlap = False
    spatial_dropout = False

    def decode(self, chromosome, lr=0.001, fp=32, type_conv='sep'):
        connections = chromosome.connections.matrix
        cnn_layers = chromosome.cnn_layers

        if self.maxpool_overlap:
            ps = 3
            st = 2
        else:
            ps = 2
            st = 2

        def decode_layer(layer, inp_):
            act_ = layer.activation
            filters = layer.filters
            k_size = layer.k_size
            
            act_aux = act_
            if act_ in ['relu', 'sigmoid', 'tanh', 'elu']:
                act_ = act_
            else:
                act_ = None
            if type_conv == 'sep':
                x_ = SeparableConv2D(filters, k_size, activation=act_, padding='same')(inp_)
            elif type_conv == 'depw':
                x_ = DepthwiseConv2D(kernel_size=k_size,depth_multiplier=2, activation=act_, padding='same')(inp_)
            elif type_conv == 'loc':
                x_ = LocallyConnected2D(filters, kernel_size=k_size, activation=act_, padding='same')(inp_)
            else:
                x_ = Conv2D(filters, kernel_size=k_size, activation=act_, padding='same')(inp_)
                
            if act_aux == 'prelu':
                x_ = PReLU()(x_)
            elif act_aux == 'leakyreLu':
                x_ = LeakyReLU()(x_)

            x_ = BatchNormalization()(x_)
            if layer.maxpool:
                x_ = MaxPooling2D(pool_size=ps, strides=st)(x_)
            if self.spatial_dropout:
                x_ = SpatialDropout2D(layer.dropout)(x_)
            else:
                x_ = Dropout(layer.dropout)(x_)

            return x_

        def count_mp(s1, s2):
            n = 0
            while True:
                if s1 == s2:
                    return n
                if FitnessSkip.maxpool_overlap:
                    s1 = int((s1 + 1) / 2 - 1)
                else:
                    s1 = int(s1/2)
                n += 1

        inp = Input(shape=self.input_shape)
        x = BatchNormalization()(inp)

        layers = []
        if len(cnn_layers) > 0:
            layers.append(decode_layer(cnn_layers[0], x))

        for block in range(connections.shape[0]):
            input_connections = []
            for input_layer in range(connections.shape[0]):
                if connections[block, input_layer] == 1:
                    input_connections.append(layers[input_layer])

            if len(input_connections) > 1:
                shapes = [l._shape_as_list()[1] for l in input_connections]
                min_shape = np.min(shapes)
                for k in range(len(input_connections)):
                    maxpool_size = count_mp(shapes[k], min_shape) + 1
                    while maxpool_size > 1:
                        input_connections[k] = MaxPooling2D(pool_size=ps, strides=st)(input_connections[k])
                        maxpool_size -= 1
                x = concatenate(input_connections)
            else:
                x = input_connections[0]
            layers.append(decode_layer(cnn_layers[block + 1], x))

        if len(layers) == 0:
            x = Flatten()(x)
        else:
            x = Flatten()(layers[-1])

        for i in range(chromosome.n_nn):
            act = chromosome.nn_layers[i].activation
            if act in ['relu', 'sigmoid', 'tanh', 'elu']:
                x = Dense(chromosome.nn_layers[i].units, activation=act)(x)
            elif act == 'prelu':
                x = Dense(chromosome.nn_layers[i].units)(x)
                x = PReLU()(x)
            else:
                x = Dense(chromosome.nn_layers[i].units)(x)
                x = LeakyReLU()(x)
            x = BatchNormalization()(x)
            x = Dropout(chromosome.nn_layers[i].dropout)(x)
        x = Dense(self.num_clases, activation='softmax')(x)

        model = Model(inputs=inp, outputs=x)
        if self.verb:
            model.summary()
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr),
                      metrics=['accuracy'])
        return model

data_folder = '/home/daniel/datasets/MNIST_variations'
command = 'python ./train_gen.py'
verbose = 1
experiments_folder = '../super_conv'
gpus = 1

In [2]:
# dataset params:
data_folder = data_folder
smooth = 0.1
classes = []

dataset = 'MRDBI'
dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
(x_train, y_train), (x_test, y_test), (x_val, y_val) = dm.load_data()


if smooth > 0:
    y_train = smooth_labels(y_train, smooth)

(9600, 28, 28, 1) train samples
(2400, 28, 28, 1) validation samples
(50000, 28, 28, 1) test samples


In [9]:
f = FitnessSkip_v2()
f.input_shape = (28, 28, 1)
f.num_clases = 10
f.verb = 0

keras.backend.clear_session()
c = get_chromosome_from_file('./models/%s' % dataset)
model = f.decode(c, type_conv='sep')

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 28, 28, 57)   123         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 28, 28, 57)   44688       separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
batch_norm

In [12]:
batch_size = 128
epochs = 25
w_epochs = epochs // 2
total_steps = int(epochs * y_train.shape[0] / batch_size)
w_steps = int(w_epochs * y_train.shape[0] / batch_size)

keras.backend.clear_session()


for conv_type in ['sep', 'depw', 'loc', 'conv2d']:
    if conv_type in ['depw', 'loc', 'sep']:
        continue
    print("Training with conv2D %s" % conv_type)
    callbacks = [LinearDecayScheduler(learning_rate_base=0.03,
                           total_steps=total_steps,
                           warmup_learning_rate=0.00003,
                           warmup_steps=w_steps,
                           hold_base_rate_steps=0, verbose=0)]
    f.spatial_dropout = False
    f.maxpool_overlap = True
    model = f.decode(c, type_conv=conv_type)
    model.summary()
    h = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=callbacks,
          verbose=1)
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])

Training with conv2D conv2d
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 57)   570         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
p_re_lu_1 (PReLU)               (None, 28, 28, 57)   44688       conv2d_1[0][0]                   
_________________________________________________________________________________

9600/9600 [==============================] - 28s 3ms/step - loss: 1.7422 - acc: 0.4499 - val_loss: 1.2339 - val_acc: 0.5771
Epoch 3/25
9600/9600 [==============================] - 28s 3ms/step - loss: 1.3130 - acc: 0.6644 - val_loss: 0.9256 - val_acc: 0.7033
Epoch 4/25
9600/9600 [==============================] - 28s 3ms/step - loss: 1.1154 - acc: 0.7592 - val_loss: 0.7280 - val_acc: 0.7683
Epoch 5/25
9600/9600 [==============================] - 27s 3ms/step - loss: 1.0161 - acc: 0.8069 - val_loss: 0.5547 - val_acc: 0.8296
Epoch 6/25
9600/9600 [==============================] - 28s 3ms/step - loss: 0.9415 - acc: 0.8430 - val_loss: 0.5959 - val_acc: 0.8067
Epoch 7/25
9600/9600 [==============================] - 28s 3ms/step - loss: 0.9109 - acc: 0.8567 - val_loss: 0.8873 - val_acc: 0.7358
Epoch 8/25
9600/9600 [==============================] - 28s 3ms/step - loss: 0.8624 - acc: 0.8825 - val_loss: 0.6806 - val_acc: 0.7942
Epoch 9/25
9600/9600 [==============================] - 28s 3ms/st

In [77]:
i = Input(shape=(224, None,3))
x = LocallyConnected2D(64, kernel_size=3, activation='relu', padding='valid')(i)
x = MaxPooling2D(3,2)(x)
model = Model(i, x)
model.summary()

ValueError: The spatial dimensions of the inputs to  a LocallyConnected2D layer should be fully-defined, but layer received the inputs shape (None, None, None, 3)

In [52]:
SeparableConv2D?
